In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import pickle

In [2]:
data=pd.read_csv("../input/records/records.csv")
all_text=data["transcription_text"]

In [3]:
data.shape

(35168, 16)

In [4]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punctuation
punct='!"#$%&\'()*+-/<=>?@[\\]^_`{|}~'
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    return s
start=time.time()
for i in tqdm(range(35168)):
    all_text[i] = process(all_text[i])
for i in tqdm(range(35168)):
    all_text[i] = " ".join(all_text[i])
end=time.time()
print(end-start)

100%|██████████| 35168/35168 [00:24<00:00, 1462.25it/s]

131.21996474266052


In [5]:
all_text

0        one hundred dollars . I never heard Bagby comp...
1        The Freedmens Bureau vs. B.M . Richardson Assa...
2        The Freedmens Bureau vs. B. M. Richardson , Co...
3        one hundred dollars , two suits of clothes wou...
4        questions , I gave answer , I whipped.— All I ...
                               ...                        
35163    648 Z Date.Name of Male.Place of Residence.Nam...
35164    650 Date . Name of Male . Name of Female . Pla...
35165    Sarah Brame Died June 25th 1847 . 1866 1847 De...
35166    Frances S. Benners Born August 30th 1795 Died ...
35167    State of North Carolina Bladen County Alfred ....
Name: transcription_text, Length: 35168, dtype: object

****

****

In [6]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 544.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=8600a4c7b1d1d43423e350aec2db0d189668538b7e75823495cb0d18a2ecf913
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
Note: you may need to restart the kernel to use updated packages.


**a function that return a dictionnary with key= cluster_id and value="number of element in this cluster"**

In [7]:
def num_of_element(clust_assign, clus_sent) :
    dic={}
    L=list(clust_assign)
    for i in range(len(clus_sent )):
        dic[i]=L.count(i)
    
    return dic
        

In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(all_text)




Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Batches:   0%|          | 0/1099 [00:00<?, ?it/s]

In [9]:
embeddings.shape

(35168, 384)

In [10]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

283.1887745857239


In [11]:
type(clustered_sentences)

dict

In [12]:
len(cluster_assignment)

35168

In [13]:
len(clustered_sentences)

2170

In [14]:
cluster_assignment

array([1774,  892,  892, ...,  177,  177, 1384])

In [15]:
dic1=num_of_element(cluster_assignment,clustered_sentences)

In [16]:
dic1[0],dic1[100],dic1[500],dic1[1500],dic1[2169]

(20, 20, 16, 12, 6)

In [17]:
l1=[dic1[i] for i in range(2170)]
mean=np.mean(l1)
median=np.median(l1)
mean , median

(16.206451612903226, 11.0)

In [18]:
with open('dic1!5distance.pkl', 'wb') as f:
    pickle.dump(dic1, f)

****

**distance_threshold=5**

In [19]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment2 = clustering_model.labels_

clustered_sentences2 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment2):
    if cluster_id not in clustered_sentences2:
        clustered_sentences2[cluster_id] = []

    clustered_sentences2[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

292.1101975440979


In [20]:
len(clustered_sentences2)

146

In [21]:
dic2=num_of_element(cluster_assignment2,clustered_sentences2)

In [22]:
dic2[0]

172

In [23]:
dic2[0],dic2[100],dic2[50],dic2[110],dic2[145]

(172, 247, 205, 142, 34)

In [24]:
l2=[dic2[i] for i in range(146)]
mean=np.mean(l2)
median=np.median(l2)
maxx=np.max(l2)
minn=np.min(l2)
(mean , median),(minn , maxx)

((240.87671232876713, 192.5), (30, 1071))

In [25]:
with open('dic5distance.pkl', 'wb') as f:
    pickle.dump(dic2, f)

****

**distance_threshold=7**

In [26]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=7) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment3 = clustering_model.labels_

clustered_sentences3 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment3):
    if cluster_id not in clustered_sentences3:
        clustered_sentences3[cluster_id] = []

    clustered_sentences3[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

272.33588123321533


In [27]:
len(clustered_sentences3)

82

In [28]:
dic3=num_of_element(cluster_assignment3,clustered_sentences3)

In [29]:
dic3[0]

702

In [30]:
dic3[0],dic3[30],dic3[50],dic3[70],dic3[81]

(702, 386, 634, 383, 221)

In [31]:
l3=[dic3[i] for i in range(82)]
mean=np.mean(l3)
median=np.median(l3)
maxx=np.max(l3)
minn=np.min(l3)
(mean , median),(minn , maxx)

((428.8780487804878, 351.5), (67, 1845))

In [32]:
with open('dic7distance.pkl', 'wb') as f:
    pickle.dump(dic3, f)

****

**distance_threshold=3**

In [33]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=3) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment4 = clustering_model.labels_

clustered_sentences4 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment4):
    if cluster_id not in clustered_sentences4:
        clustered_sentences4[cluster_id] = []

    clustered_sentences4[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

268.3536865711212


In [34]:
len(clustered_sentences4)

407

In [35]:
dic4=num_of_element(cluster_assignment4,clustered_sentences4)

In [36]:
dic4[0]

92

In [37]:
dic4[0],dic4[100],dic4[250],dic4[300],dic4[406]

(92, 87, 66, 59, 21)

In [38]:
l4=[dic4[i] for i in range(407)]
mean=np.mean(l4)
median=np.median(l4)
maxx=np.max(l4)
minn=np.min(l4)
(mean , median),(minn , maxx)

((86.40786240786241, 69.0), (11, 487))

In [39]:
with open('dic3distance.pkl', 'wb') as f:
    pickle.dump(dic4, f)

****

**distance_threshold=3.5**

In [40]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=3.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment5 = clustering_model.labels_

clustered_sentences5 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment5):
    if cluster_id not in clustered_sentences5:
        clustered_sentences5[cluster_id] = []

    clustered_sentences5[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

273.2302041053772


In [41]:
len(clustered_sentences5)

309

In [42]:
dic5=num_of_element(cluster_assignment5,clustered_sentences5)

In [43]:
dic5[0]

153

In [44]:
dic5[0],dic5[100],dic5[150],dic5[250],dic5[300]

(153, 128, 144, 66, 59)

In [45]:
l5=[dic5[i] for i in range(309)]
mean=np.mean(l5)
median=np.median(l5)
maxx=np.max(l5)
minn=np.min(l5)
(mean , median),(minn , maxx)

((113.81229773462783, 94.0), (14, 569))

In [46]:
with open('dic3!5distance.pkl', 'wb') as f:
    pickle.dump(dic5, f)

****

In [47]:
start=time.time()
# Normalize the embeddings to unit length
embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=4) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings)
cluster_assignment6 = clustering_model.labels_

clustered_sentences6 = {}
for sentence_id, cluster_id in enumerate(cluster_assignment6):
    if cluster_id not in clustered_sentences6:
        clustered_sentences6[cluster_id] = []

    clustered_sentences6[cluster_id].append(all_text[sentence_id])
    
end=time.time()
print(end-start)

275.0343773365021


In [48]:
len(clustered_sentences6)

237

In [49]:
dic6=num_of_element(cluster_assignment6,clustered_sentences6)

In [50]:
dic6[0]

134

In [51]:
dic6[0],dic6[70],dic6[110],dic6[180],dic6[230]

(134, 60, 54, 111, 40)

In [52]:
with open('dic4distance.pkl', 'wb') as f:
    pickle.dump(dic6, f)

In [53]:
l6=[dic6[i] for i in range(237)]

In [54]:
mean=np.mean(l6)
median=np.median(l6)
maxx=np.max(l6)
minn=np.min(l6)
(mean , median),(minn , maxx)

((148.38818565400845, 113.0), (14, 572))

In [55]:
l6[:10]

[134, 266, 135, 343, 293, 264, 338, 149, 136, 182]